In [ ]:
import os
import requests

download_name = "dataset.zip"
if not os.path.exists(download_name):
    import requests
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/inlp_repo/main/labs/opj_v07_analiza_sentimenta/{download_name}")
    with open(download_name, "wb") as fp:
        fp.write(response.content)
    response.close()

if not os.path.exists("imdb1"):
    from zipfile import ZipFile
    with ZipFile(download_name) as zf:
        zf.extractall(path=".")


name = "opj_v07_test.py"
if not os.path.exists(name):
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/inlp_repo/main/labs/opj_v07_analiza_sentimenta/{name}")
    with open(name, "wb") as fp:
        fp.write(response.content)
    response.close()


# 7. Analiza sentimenta

In [ ]:
import os
import math

from opj_v07_test import *

DATA_PATH = 'imdb1/'
STOPWORD_FILENAME = 'english.stop'


## 7.1 Izgradnja modela dokumenata

Funkcija **make_datasets()** ima sljedeće ulazne i izlazne parametre
* ulaz:
    * **data_path** putanja do datoteka
* izlaz:
    * lista oblika  

<pre>
0: train: pos: [filename, filename, ...]
          neg: [filename, filename, ...]
   test:  pos:  [filename, filename, ...]  
          neg:  [filename, filename, ...]  
1: train: pos: [filename, filename, ...]  
          neg: [filename, filename, ...]  
   test:  pos:  [filename, filename, ...]  
          neg:  [filename, filename, ...]  
...  
...
</pre>

Svaki element liste se zove "preklop" (engl. fold) i sastoji se od dokumenata za treniranje i testiranje. 
Preklop je rječnik sa dva ključa "train" i "test" čije su vrijednosti rječnik s dva ključa "pos" i "neg" čije su vrijednosti lista datoteka koje su "pos" ili "neg" kritike.

In [ ]:
def make_datasets(data_path, numfolds = 10):
    """
    ulaz:
    -data_path: putanja do datoteka
    izlaz:
    -lista oblika
    0: train: pos: [filename, filename, ...]
              neg: [filename, filename, ...]
       test: pos:  [filename, filename, ...]
             neg:  [filename, filename, ...]
    1: train: pos: [filename, filename, ...]
              neg: [filename, filename, ...]
       test: pos:  [filename, filename, ...]
             neg:  [filename, filename, ...]
    ...
    ...
    Svaki element liste je rječnik sa dva ključa "train" i "test"
    čije su vrijednosti rječnik s dva ključa "pos" i "neg" čije su
    vrijednosti lista datoteka koje su "pos" ili "neg" kritike
    """
    klasses = [klass for klass in os.listdir(data_path) if not klass.startswith('.')]
    datasets = []
    filenames = {klass: sorted(os.listdir(data_path + klass + '/')) for klass in klasses}
    for fold in range(numfolds):
        trains = {klass: [] for klass in klasses}
        tests = {klass: [] for klass in klasses}
        for klass in klasses:
            for filename in filenames[klass]:
                if filename[2] == str(fold):
                    tests[klass].append(data_path + klass + '/' + filename)
                else:
                    trains[klass].append(data_path + klass + '/' + filename)
        datasets.append({'train': trains, 'test': tests})
    return datasets

datasets = make_datasets(DATA_PATH)
datasets

## 7.1 Učitavanje stop riječi i dokumenata

Napravi funkciju **read_stopwords()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **filename** datoteka
* izlaz:
    * skup stop riječi iz datoteke

In [ ]:
def read_stopwords(filename):
    """
    ulaz:
    -filename: datoteka

    izlaz:
    -skup stop riječi iz datoteke
    """
    return

testname(read_stopwords)
stopwords = test(read_stopwords, (STOPWORD_FILENAME,), {'nine', 'willing', 'inc', 'because', 'edu', 'whenever', 'well', 'hence', 'may', 'seven', 'sometime', 'becomes', 'among', 'had', 'lest', 'containing', 'yourself', 'of', 'theirs', "i'll", 'said', 'down', 'p', 'hereby', 'sub', 'then', 'there', 'qv', 'nothing', 'is', 'tries', 'new', 'x', 'everywhere', 'otherwise', 'com', 'presumably', 'amongst', 'become', 'were', 'they', 'her', 'also', 'hers', 'second', 'tried', 'gone', 'two', 'wants', 'above', 'although', 'much', 'rd', 'f', 'was', 'have', 'has', 'just', 'thorough', 'took', 'particular', 'together', 'up', 'became', 'mean', 'anybody', "they're", 'former', 'only', 'follows', 'right', 'think', 'especially', 'we', 'not', 'concerning', 'which', 'despite', 'tends', 'course', "aren't", 'used', 'cant', 'formerly', 'anyways', 'usually', 'your', 'goes', 'having', 'here', 'nor', "what's", "c's", 'reasonably', 'a', 'see', 'theres', 'meanwhile', "we'll", 'how', "you'd", 'third', 'contain', 'brief', 'becoming', 'asking', 'r', 'liked', 'novel', 'me', 'for', 'its', 'unfortunately', 'elsewhere', 'uses', 'are', 'whereupon', 'overall', 'know', 'consider', 'help', 'uucp', 'furthermore', 'enough', 'according', 'anyway', 'latter', 'far', 'truly', "we've", 'definitely', 'keep', 'yet', 'mostly', 'behind', 'un', 'serious', 'doing', 'probably', 'though', 'what', 'must', 'our', 'say', 'contains', 'upon', 'provides', 'more', "don't", 'per', "t's", 'thereupon', 'unlikely', 'going', 'always', 'anything', 'currently', 'still', 'associated', 'obviously', 'and', 'very', 'described', 'better', 'do', "you'll", 'him', 'looks', 'hardly', "haven't", 'k', "they've", 'seriously', 'thereafter', 'nd', 'particularly', 'be', 'whom', 'out', 'like', 'welcome', 'able', 'go', 'gives', 'seeming', 'trying', 'anyone', 'each', 'inner', 'namely', 'he', 'to', 'thank', 'beyond', 'herein', 'm', 'useful', 'various', 'wherever', 'thus', 'l', 'ask', 'within', 'merely', "shouldn't", "that's", 'sometimes', "they'll", 'wherein', "where's", "ain't", 'gotten', 'somehow', 'ignored', 'therein', 'ever', 'yours', 'other', 'let', 'certain', 'changes', 'whereby', 'hither', 'allow', 'could', 'respectively', 'inward', 'wonder', 'under', 'someone', 'therefore', 'necessary', 'etc', 'in', 'least', 'awfully', 'relatively', 'shall', 'corresponding', 'should', 'appear', 'need', 'whither', 'non', 'looking', 'get', 'v', 'sorry', "wouldn't", 'whereas', 'lately', 'n', 'says', 'wish', 'soon', 'six', 'himself', 'entirely', 'got', "i'd", 'even', 'example', 'e', 'self', 'vs', 'several', 'considering', 'however', 'such', 'why', 'getting', 'happens', 'okay', 'some', "there's", 'towards', "hasn't", 'sent', 'th', "we're", 'few', 'seemed', 'believe', 'four', 'b', 'whereafter', 'three', 'who', 'aside', 'does', 'or', 'actually', 'placed', "doesn't", 'further', 'latterly', 'once', 'maybe', 'one', 'herself', 'indicate', 'normally', 'regardless', 'ones', 'already', 'name', 'eight', 'so', 'their', 'ex', 'after', 'while', 'u', "you've", 'available', 'g', 'keeps', 'unto', 'afterwards', 'near', 'later', 'those', 'certainly', 'j', 'o', 'went', 'since', 'plus', 'whole', 'less', 'consequently', 'now', 'd', 'known', 'thanx', 'w', 'yes', 'oh', 'alone', 'my', 'us', 'neither', 'below', 'taken', 'hereafter', 'nowhere', 'whence', 'off', 'where', 'ourselves', 'regarding', 'given', 'by', 'except', 'immediate', "isn't", 'forth', 'over', 'else', 'try', 'everything', 'via', 'inasmuch', 'ours', 'toward', 'often', 'y', 'until', 'using', 'perhaps', 'them', 'about', 'seems', 'gets', 'specified', 'knows', 'seen', 'without', 'cannot', 'appropriate', 'every', 'clearly', 'both', 'downwards', "a's", 'everybody', 'accordingly', 'cause', 'old', 'with', 'tell', 'i', 'anyhow', 'besides', "who's", 'that', 'following', 'can', 'itself', 'q', 'thru', 'these', 'little', 'the', 'viz', 'hi', 'first', 'against', 'somewhat', "weren't", 'next', 'kept', 'thanks', 'sup', 'themselves', "wasn't", 'sure', "he's", 'done', 'allows', 'fifth', 'twice', 'across', 'apart', 'own', 'before', 'at', 'nearly', 'during', 'ltd', 'noone', 'causes', 'from', 'followed', 'away', "couldn't", 'best', 'last', 'most', 'nobody', 'almost', "won't", 'either', 'you', "let's", 'many', 'thence', 'hereupon', 'exactly', 'somewhere', 'please', 'anywhere', 'another', 'insofar', 'quite', 'way', 'appreciate', 'ok', 'regards', 'outside', 'five', 'between', 'ought', 'whoever', 'specify', 'would', 'throughout', 'instead', "here's", 'no', 'z', 'through', 'c', 'secondly', "c'mon", 'she', 'really', "hadn't", 'into', 'if', 'ie', 'want', 'same', 'whether', 'all', 'any', 'indicated', "i'm", 'zero', 's', 'will', 'thats', 'myself', 'possible', 'sensible', 'co', 'on', 'but', 'an', 'again', 'take', 'value', 'et', 'h', 'mainly', 'specifying', 'beforehand', 'as', 'around', 'thoroughly', 'beside', 'been', 'moreover', 'different', 'somebody', 'might', "we'd", 'likely', 'indeed', 'eg', "they'd", 'rather', 'nevertheless', 'others', "can't", "you're", 'selves', 'howbeit', 'hopefully', 'did', 'whose', 'when', 'came', 'his', 'never', 'whatever', 'along', "it'll", 'seeing', 'this', 'it', "it'd", 'am', 'come', 'unless', 'everyone', "i've", 'something', 'than', 'use', 're', 'hello', 'que', 'thereby', 'indicates', 'none', 't', 'seem', 'needs', 'yourselves', "didn't", 'onto', 'greetings', 'saying', 'comes', "it's", 'being', 'saw', 'look', 'too'})

### Dokument

Funkcija **read_document()** ima sljedeće ulazne i izlazne parametre
* ulaz:
    * **filename** datoteka dokumenta
    * stopwords: skup stop riječi
* izlaz:
    * skup riječi iz datoteke koje nisu stop riječi
    
Ova funkcija će odvojiti riječi iz teksta datoteke po praznim redovima i razmacima (koristiti `split()` metodu stringa)

In [ ]:
def read_document(filename, stopwords = set()):
    """
    ulaz:
    -filename: datoteka dokumenta
    -stopwords: skup stop riječi

    izlaz:
    -skup riječi iz datoteke koje nisu u stopwords
    """
    return

testname(read_document)
_ = test(read_document, (datasets[0]['train']['pos'][0], ), set(['james', 'l', '.', 'brooks', ',', 'one', 'of', 'the', 'developers', 'of', 'the', 'simpsons', 'and', 'director', 'of', 'broadcast', 'news', ',', 'returns', 'to', 'the', 'big', 'screen', 'with', 'this', 'entertaining', ',', 'if', 'slightly', 'flawed', 'comedy', '.', 'nicholson', 'plays', 'melvin', 'udall', ',', 'probably', 'the', 'most', 'horrible', 'person', 'ever', 'on', 'the', 'screen', '.', "he's", 'racist', ',', 'homophobic', ',', 'and', 'never', 'has', 'a', 'good', 'word', 'to', 'say', 'to', 'anyone', '.', 'so', ',', 'nobody', 'talks', 'to', 'him', ',', 'except', 'waitress', 'carol', 'conelly', '(', 't', '.', 'v', 'sitcom', 'star', 'hunt', ',', 'who', 'was', 'last', 'seen', 'in', 'twister', ',', '1996', ')', '.', 'naturally', ',', 'udall', ',', 'conelly', 'and', 'gay', 'neighbor', 'simon', 'bishop', '(', 'kinnear', ')', 'who', 'nicholson', 'hates', ',', 'all', 'hit', 'it', 'off', 'in', 'the', 'end', '.', 'like', 'good', 'will', 'hunting', '(', '1997', ')', 'and', 'titanic', '(', '1997', ')', ',', 'even', 'though', 'the', 'outcome', 'is', 'completely', 'obvious', ',', 'as', 'good', 'as', 'it', 'gets', 'is', 'an', 'enjoyable', ',', 'funny', 'and', 'warm', 'comedy', '.', 'nicholson', 'is', 'hilarious', 'as', 'melvin', ',', 'churning', 'out', 'insults', 'with', 'superb', 'relish', '.', 'only', 'nicholson', 'could', 'get', 'away', 'with', 'the', 'lines', 'that', 'melvin', 'delivers', '.', 'hunt', 'is', 'also', 'good', 'as', 'waitress', 'carol', ',', 'and', 'easily', 'rises', 'to', 'the', 'challenge', 'of', 'nicholson', '.', "there's", 'also', '(', 'thankfully', ')', 'a', 'bit', 'of', 'chemistry', 'between', 'them', '.', 'kinnear', ',', 'as', 'the', 'gay', 'neighbor', ',', 'seems', 'to', 'have', 'a', 'slightly', 'underwritten', 'role', ',', "he's", 'more', 'of', 'a', 'plot', 'convience', 'than', 'a', 'character', '.', 'although', 'his', 'performance', 'is', 'good', ',', 'his', 'character', 'just', 'seems', 'to', 'exist', 'to', 'help', 'melvin', 'and', 'carol', 'come', 'together', '.', 'in', 'fact', ',', 'the', 'scene', 'stealer', 'is', "simon's", 'dog', ',', 'who', 'is', 'funnier', 'than', 'nicholson', '.', 'but', 'then', 'again', ',', 'pets', 'are', 'always', 'cute', 'on', 'screen', '.', 'providing', 'solid', 'support', 'is', 'cuba', 'gooding', ',', 'jnr', '(', 'jerry', 'maguire', ',', '1996', ')', 'and', 'yeardly', 'smith', '(', 'who', 'is', 'the', 'voice', 'of', 'lisa', 'simpsons', 'in', 'the', 'simpsons', ')', 'although', 'gooding', "isn't", 'as', 'good', 'as', 'is', 'character', 'in', 'maguire', ',', 'he', 'is', 'still', 'fun', '.', 'he', 'overacts', 'a', 'little', ',', 'but', 'not', 'so', 'much', 'as', 'to', 'be', 'annoying', '.', 'smith', 'is', 'also', 'good', ',', 'although', 'she', 'has', 'a', 'fairly', 'small', 'role', '.', 'even', 'director', 'lawrence', 'kasdan', '(', 'body', 'heat', ',', '1981', ')', 'makes', 'an', 'appearance', 'as', 'a', 'doctor', '.', 'but', 'this', 'is', 'primarily', 'nicholsons', 'film', ',', 'and', 'every', 'scene', "he's", 'in', ',', "he's", 'steals', 'it', '.', "he's", 'character', 'is', 'so', 'hateful', ',', 'though', ',', "it's", 'amazing', 'that', 'anyone', 'talks', 'to', 'him', 'at', 'all', ',', 'especially', 'carol', '.', 'and', 'this', 'is', 'the', 'films', 'main', 'problem', '.', "it's", 'totally', 'unbelievable', 'that', 'carol', 'would', 'ever', 'consider', 'liking', 'melvin', '.', 'she', "doesn't", 'fall', 'in', 'love', 'with', 'him', 'naturally', ',', 'the', 'film', 'forces', 'her', 'to', 'fall', 'in', 'love', 'with', 'him', '.', 'also', ',', 'melvins', 'character', 'seems', 'to', 'go', 'too', 'nice', ',', 'too', 'quickly', '.', 'i', 'would', 'doubt', 'anyone', 'with', 'a', 'character', 'like', 'melvins', 'would', 'be', 'able', 'to', 'turn', 'back', 'to', 'a', 'nice', ',', 'loving', 'person', '.', 'it', 'would', 'take', 'a', 'helluva', 'long', 'time', ',', 'much', 'longer', 'than', 'this', 'film', 'would', 'like', 'to', 'make', 'out', '.', 'brooks', 'direction', 'is', 'good', ',', 'though', ',', 'if', 'a', 'bit', 'average', ',', 'but', 'he', 'usually', 'manages', 'to', 'get', 'an', 'emotion', 'out', 'of', 'the', 'audience', '.', 'he', 'handles', 'the', 'comedy', 'scenes', 'better', 'than', 'the', 'sentimental', 'ones', '(', 'he', 'tends', 'to', 'pile', 'on', 'to', 'much', 'schmaltz', ')', 'but', 'generally', "he's", 'good', '.', "there's", 'also', 'a', 'nice', 'soundtrack', 'by', 'veteran', 'composer', 'hans', 'zimmer', '.', 'but', ',', 'generally', ',', 'as', 'good', 'as', 'it', 'gets', 'achieves', 'what', 'it', 'sets', 'out', 'to', 'do', ',', 'which', 'is', 'to', 'make', 'the', 'audience', 'feel', 'good', 'by', 'the', 'end', 'of', 'the', 'movie', '.', 'the', 'movie', 'is', 'a', 'bit', 'overlong', ',', 'but', 'nicholson', 'is', 'such', 'good', 'fun', 'that', 'the', 'running', 'time', 'passes', 'by', 'pretty', 'quickly', '.', 'overall', ',', 'as', 'good', 'as', 'it', 'gets', 'is', 'a', 'fun', 'movie', ',', 'even', 'though', 'it', 'may', 'be', 'unbelivable', ',', 'and', 'certainly', 'worth', 'seeing', '(', 'if', 'just', 'for', 'jack', 'nicholsons', 'performance', '.', ')', 'not', 'quite', 'as', 'good', 'as', 'it', 'gets', '(', 'pardon', 'the', 'bad', 'joke', ')', ',', 'but', 'still', 'good', 'fun', '.']))
_ = test(read_document, (datasets[0]['train']['pos'][0], stopwords), set(['james', '.', 'brooks', ',', 'developers', 'simpsons', 'director', 'broadcast', 'news', ',', 'returns', 'big', 'screen', 'entertaining', ',', 'slightly', 'flawed', 'comedy', '.', 'nicholson', 'plays', 'melvin', 'udall', ',', 'horrible', 'person', 'screen', '.', 'racist', ',', 'homophobic', ',', 'good', 'word', '.', ',', 'talks', ',', 'waitress', 'carol', 'conelly', '(', '.', 'sitcom', 'star', 'hunt', ',', 'twister', ',', '1996', ')', '.', 'naturally', ',', 'udall', ',', 'conelly', 'gay', 'neighbor', 'simon', 'bishop', '(', 'kinnear', ')', 'nicholson', 'hates', ',', 'hit', 'end', '.', 'good', 'hunting', '(', '1997', ')', 'titanic', '(', '1997', ')', ',', 'outcome', 'completely', 'obvious', ',', 'good', 'enjoyable', ',', 'funny', 'warm', 'comedy', '.', 'nicholson', 'hilarious', 'melvin', ',', 'churning', 'insults', 'superb', 'relish', '.', 'nicholson', 'lines', 'melvin', 'delivers', '.', 'hunt', 'good', 'waitress', 'carol', ',', 'easily', 'rises', 'challenge', 'nicholson', '.', '(', 'thankfully', ')', 'bit', 'chemistry', '.', 'kinnear', ',', 'gay', 'neighbor', ',', 'slightly', 'underwritten', 'role', ',', 'plot', 'convience', 'character', '.', 'performance', 'good', ',', 'character', 'exist', 'melvin', 'carol', '.', 'fact', ',', 'scene', 'stealer', "simon's", 'dog', ',', 'funnier', 'nicholson', '.', ',', 'pets', 'cute', 'screen', '.', 'providing', 'solid', 'support', 'cuba', 'gooding', ',', 'jnr', '(', 'jerry', 'maguire', ',', '1996', ')', 'yeardly', 'smith', '(', 'voice', 'lisa', 'simpsons', 'simpsons', ')', 'gooding', 'good', 'character', 'maguire', ',', 'fun', '.', 'overacts', ',', 'annoying', '.', 'smith', 'good', ',', 'fairly', 'small', 'role', '.', 'director', 'lawrence', 'kasdan', '(', 'body', 'heat', ',', '1981', ')', 'makes', 'appearance', 'doctor', '.', 'primarily', 'nicholsons', 'film', ',', 'scene', ',', 'steals', '.', 'character', 'hateful', ',', ',', 'amazing', 'talks', ',', 'carol', '.', 'films', 'main', 'problem', '.', 'totally', 'unbelievable', 'carol', 'liking', 'melvin', '.', 'fall', 'love', 'naturally', ',', 'film', 'forces', 'fall', 'love', '.', ',', 'melvins', 'character', 'nice', ',', 'quickly', '.', 'doubt', 'character', 'melvins', 'turn', 'back', 'nice', ',', 'loving', 'person', '.', 'helluva', 'long', 'time', ',', 'longer', 'film', 'make', '.', 'brooks', 'direction', 'good', ',', ',', 'bit', 'average', ',', 'manages', 'emotion', 'audience', '.', 'handles', 'comedy', 'scenes', 'sentimental', '(', 'pile', 'schmaltz', ')', 'generally', 'good', '.', 'nice', 'soundtrack', 'veteran', 'composer', 'hans', 'zimmer', '.', ',', 'generally', ',', 'good', 'achieves', 'sets', ',', 'make', 'audience', 'feel', 'good', 'end', 'movie', '.', 'movie', 'bit', 'overlong', ',', 'nicholson', 'good', 'fun', 'running', 'time', 'passes', 'pretty', 'quickly', '.', ',', 'good', 'fun', 'movie', ',', 'unbelivable', ',', 'worth', '(', 'jack', 'nicholsons', 'performance', '.', ')', 'good', '(', 'pardon', 'bad', 'joke', ')', ',', 'good', 'fun', '.']))


## 7.2 Učenje klasifikatora

Napravi funkciju **train()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **testset** rječnik čiji ključevi su klase, a vrijednosti putanje do datoteka
    * **stopwords** skup stop-riječi
* izlaz:
    * **prior** je rječnik čiji su ključevi klase, a vrijednosti prior vjerojatnosti za svaku klasu
    * **megadoc** je rječnik čiji su ključevi klase, a vrijednosti unigram s frekvencijama riječi
    * **vocabulary** je skup svih riječi iz svih megadokumenata


$$P(c) = \frac{Nc}{N}$$

* $c$- klasa  
* $Nc$ - broj dokumenata klase c  
* $N$ - broj svih dokumenata


In [ ]:
def train(trainset, stopwords):
    return
        
testname(train)
model = test(train, (datasets[1]["train"], stopwords), test_train_output)

## 7.3 Klasifikacija

Napravi funkciju **classify()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **model** trenirani model kojeg vraća funkcija `train()`
    * **filename** datoteka koja se klasificira
    * **stopwords** stop-riječi
* izlaz:
    * **klasa**

Vjerojatnost dokumenta $d$ za klasu $c$ se računa po Naivnom Bayesu uz dodaj-1 izglađivanje

$$ P(d|c) = p(c) \prod_{w \in d}{p(w|c)}$$ 

gdje je

$$ P(w|c) = \frac{br(w,c) + 1}{br(c) + |V|}$$

Na kraju, odabrana klasa se traži po

$$ c_{max} = \text{argmax}_{c \in C} P(d | c)$$

Radi preciznosti, koristiti logaritamski prostor za izračun $P(d|c)$ odnosno

$$ log(P(d|c)) = log(p(c)) + \sum_{w \in d}{log(p(w|c))} $$


In [ ]:
def classify(model, filename, stopwords):
    """
    ulaz:
    -train_model: trenirani model kojeg vraća funkcija build_train_model()

    izlaz:
    -megadoc_model: je rječnik čiji su ključevi klase, a vrijednosti megadokumenti kao unigrami
    nastali "spajanjem" svih unigrama iz treniranog modela za određenu klasu
    """
    return
        

testname(classify)
_ = test(classify, (model, datasets[1]["test"]["pos"][0], stopwords), "pos")
_ = test(classify, (model, datasets[1]["test"]["neg"][0], stopwords), "neg")


## 7.4 Unakrsna validacija

Funkcija **cross_validate()** vrši unakrsnu validaciju.
Za svaki "savijutak" (engl. fold) će se trenirati klasifikator i testirati preciznost.

In [ ]:
def cross_validate(has_stopwords, counts):
    """
    ulaz:
    -has_stopwords: ima ili nema stop riječi
    -counts: lista očekivanog broja točno klasificiranih dokumenata za
             svaku unakrsnu podjelu podataka

    Koraci:
    1. Pripremi unakrsnu validaciju
    2. Ako ima stop riječi onda ih učitaj
    3. Za svaku unakrsnu podjelu podataka:
       3.1. Treniraj klasifikator na podacima za trening
       3.2. Klasificiraj na podacima za testiranje
       3.3. Izračunaj točnost klasifikatora za unakrsnu podjelu
    4. Izračunaj ukupnu točnost klasifikatora
    """
    print('[INFO] Preparing datasets...')
    datasets = make_datasets(DATA_PATH)
    if has_stopwords:
        print('[INFO] Reading stopwords...')
        stopwords = read_stopwords(STOPWORD_FILENAME)
    else:
        print("[INFO] No stopwords...")
        stopwords = set()
    print('[INFO] Training and classifying...')

    total_good, total_all = 0, 0
    for fold, dataset in enumerate(datasets):
        print("[TRAIN] Fold", fold)
        trainset, testset = dataset['train'], dataset['test']
        
        model = train(trainset, stopwords)
        print("[EVALUATE] Fold", fold)
        
        # evaluating
        count_good, count_all = 0, 0
        for klass, filenames in testset.items():
            counter = len(filenames)
            for filename in filenames:
                predicted_klass = classify(model, filename, stopwords)
                count_good += 1 if predicted_klass == klass else 0
                count_all += 1
                counter -= 1
                if counter % 10 == 0:
                    i = counter // 10
                    j = 10 - i
                    print(klass, "[" + ("=" * j) + ("." * i) + "]", end="\r")
            print()
        total_good += count_good
        total_all += count_all

        success = count_good == counts[fold]
        success_txt = ' OK ' if success else ' X  '

        print(f'[{success_txt}] Fold {fold}: {count_good}/{count_all}\t{count_good/count_all * 100:.1f}%')
        print()

    print(f'[INFO] Accuracy {total_good}/{total_all}\t{total_good/total_all*100:.1f}%')

cross_validate(False, [161,168,167,165,167,165,169,167,158,171])

cross_validate(True, [162,168,168,164,167,166,168,168,153,170])